# Prepare topology and starting file

Note: There is nothing special about this peptide, it is a segment of [Trp-Cage Miniprotein](http://www.rcsb.org/pdb/explore/explore.do?pdbId=1l2y).

If you are using GPU, you can use whole protein

This tutorial use [GBNeck2 + ff14SBonlysc](http://pubs.acs.org/doi/abs/10.1021/ja5032776)

In [44]:
%%file leap.in
source leaprc.protein.ff14SBonlysc
x = sequence {NASN LEU TYR CILE}
set default PBradii mbondi3 
saveamberparm x peptide.top peptide.crd
savepdb x peptide.pdb 
quit

Overwriting leap.in


In [45]:
!$AMBERHOME/bin/tleap -f leap.in > leap.log

In [46]:
!ls peptide.*

peptide.crd  peptide.pdb  peptide.top


# Run minimization

you can use any text editor to make min.in but we can do this in this notebook by `%%file` command

In [47]:
%%file min.in
Stage 1 - minimisation of TC5b
&cntrl
  imin=1, maxcyc=1000, ncyc=500,
  cut=999.,
  igb=8,
  ntb=0,
  ntpr=100
/



Overwriting min.in


In [48]:
!$AMBERHOME/bin/sander -O -i min.in -p peptide.top -c peptide.crd -r min.rst7

# Real time visualizing simulation

In [49]:
# used for reading coordinates from rst7 file
# pytraj is available in AmberTools16
import pytraj as pt

# use nglview for visualizing this notebook
# it's available in https://github.com/arose/nglview
import nglview as nv

In [50]:
# load minimized struture and use it as reference for superposing
ref = pt.load('min.rst7', top='peptide.top')
ref

pytraj.Trajectory, 1 frames: 
Size: 0.000002 (GB)
<Topology: 76 atoms, 4 residues, 1 mols, non-PBC>
           

In [51]:
# load coordinates restart file (md.rst)
def get_coordinates():
    # return coordinates in restart file
    traj = pt.load('md.rst', top=ref.top)
    
    # perform superpose to min.rst7 to remove translation and rotation
    # superpose is similiar to cpptraj's rms
    traj.superpose(ref=ref, mask='@CA,N,O')
    return traj.xyz[0]

In [52]:
# view starting structure (min.rst7)
view = nv.show_pytraj(ref)
view

In [53]:
view.clear_representations()
view.add_ball_and_stick()
view.add_cartoon()

In [54]:
def update_view(view, delay=1):
    # update coordinates every `delay` second
    from time import sleep
    coordinates = get_coordinates()
    view.coordinates_dict = {0: coordinates}
    sleep(delay)

In [55]:
%%file md.in
md.in
&cntrl
       imin = 0, nstlim = 1000000, dt = 0.002,
       ntx = 1, irest = 0, ig = -1,
       ntt=3, tempi = 300.0, temp0 = 300.0, gamma_ln = 1.0,
       ntc = 2, ntf = 2,
       ntwx = 25000, ntwe = 0, ntwr = 100, ntpr = 25000,
       cut = 999.0,
       ntb = 0,
       igb = 8, gbsa = 0,
       nscm = 25000,
       ioutfm = 1,
/



Overwriting md.in


In [56]:
def run_md():
    # run md simulation via sander
    import os
    import subprocess
    
    # you can use sander.MPI, pmemd, pmemd.MPI, pmemd.cuda too
    # need to provide absolute directory for sander
    sander = os.getenv('AMBERHOME') + '/bin/sander'
    cm = "sander -O -i md.in -o md.out -p peptide.top -c min.rst7 -r md.rst -x md.nc".format(sander=sander)
    
    # we will run sander in background so we can keep using this notebook
    print('running {}'.format(cm))
    return subprocess.Popen(cm.split())

In [57]:
# run MD simulation in background so we can keep using this notebook
process = run_md()

running sander -O -i md.in -o md.out -p peptide.top -c min.rst7 -r md.rst -x md.nc


In [58]:
# continously update coordinates
# To interupt, you need to choose Kernel --> Interupt (top page)

# to rerun, you need to run previous command process = run_md()
while True:
    update_view(view, 0.1)
    
# While this cell is running, scroll up to see your real time animation. Enjoy

KeyboardInterrupt: 

In [28]:
# to kill MD simulation (sander), run this cell
process.kill()